## 신경망 학습

신경망이 데이터에서 학습한다는 것은 가중치 매개변수(weight와 bias)의 값을 데이터를 보고 자동으로 결정한다는 뜻이다.  
(우리가 신경망의 가중치 매개변수를 수작업으로 설정하지 않아도 되는 이유)  

> 딥러닝을 종단간 기계학습 ***end-to-end machine learning*** 이라고도 하며, 이는 데이터 입력에서 목표한 결과를 **사람의 개입없이** 얻는다는 뜻이다.

보통은 훈련 데이터와 시험 데이터를 나눠서 진행하는데, 우리가 궁극적으로 원하는 것이 범용적으로 사용할 수 있는 모델이기 때문이다.  
범용 능력은 아직 보지 못한 데이터로도 문제를 올바르게 풀어내는 능력을 뜻한다.  

### 오버 피팅
손글자 숫자 인식의 최종 결과는 택배에서 우편 번호를 자동으로 판독하는 시스템에 쓰일지 모른다!  
누구인지 모르는 수 많은 사람들이 쓴 글자를 인식하는 능력이 중요한 것.  
즉, 범용성이 떨어지게 되면 훈련 데이터는 잘 판별하지만 테스트 데이터(실전)에서는 제대로 판별하지 못하는 경우를 ***오버피팅***이라 한다.

### 손실 함수
손실 함수는 신경망 성능의 나쁨을 나타내는 지표로, 현재 신경망이 훈련 데이터를 얼마나 잘처리하지 못하느냐를 나타낸다.

### 평균 제곱 오차 - Mean Squared Error(MSE)

<img src="img/deep_learning_images/e_4.1.png" height=224 width=224>

$y_k$는 신경망이 추정한 값(출력값), $t_k$는 정답 레이블, k는 데이터의 차원수를 나타낸다.

In [3]:
import numpy as np

In [10]:
y = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]) # softmax의 출력값, 인덱스 2가 가장 확률값이 높음
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0]) # 정답은 2

In [11]:
def mean_squared_error(y, t):
    return 0.5 * sum((y - t)**2)

In [12]:
mean_squared_error(y, t)

0.09750000000000003

In [13]:
y2 = np.array([0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]) # 출력값의 인덱스 7번이 가장 높음
mean_squared_error(y2, t)

0.5974999999999999

위 코드에서 볼 수 있듯 y가 y2보다 손실 함수 값이 더 작으므로, ***오차가 더 작다로 볼 수 있다.(정답에 더 가깝다)***

### 교체 엔트로피 - Cross Entropy Error
<img src="img/deep_learning_images/e_4.2.png" width=224 height=224>

$log$는 밑이 $e$인 자연로그이며 $y_k$는 신경망의 출력, $t_k$는 정답레이블.  
구현에서 delta값을 더해주는 이유는 ```np.log```함수는 0을 입력하면 마이너스 무한대를 뜻하는 -inf가 되어 더 이상 계산을 진행할 수 없다.

In [16]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -(sum(t * (np.log(y + delta))))

In [21]:
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
y = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])

cross_entropy_error(y, t)

0.510825457099338

In [22]:
y_test1 = np.array([0.1, 0.05, 0.8, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
cross_entropy_error(y_test1, t)

0.22314342631421757

수식에서의 $t_k$는 One-hot-encoding 된 벡터이기 때문에, 정답이 아닌 다른 인덱스는 0이고 예측값 벡터에 곱해지면 정답에 해당하는 인덱스의 확률값만 남게 된다.  
즉, 정답일 때의 신경망 출력값의 자연로그를 계산하는 식이된다.  

In [26]:
t2 = np.array([0, 1])
y2 = np.array([0.1, 0.7])
delta2 = 1e-7

-(t2 * (np.log(y2 + delta2)))

array([0.       , 0.3566748])

이렇게 정답에 해당하는 출력(확률)이 커질수록 자연로그의 y값은 0에 다가가고, 출력이 1일 때는 y값은 0이된다.  
또한 출력이 작아질수록 오차는 커진다.

<img src="img/deep_learning_images/fig_4-3.png" width=448 height=448>

위에서 본 손실함수와 훈련 데이터에 관한 관계를 정리하면,  
훈련 데이터에 대한 손실 함수의 값을 구하고, 그 값을 최대한 줄여주는 매개변수를 찾아낸다.

이렇게 하려면 모든 훈련 데이터를 대상으로 손실 함수 값을 구해야 한다. 훈련 데이터 100개가 있으면 그로부터 계산한 100개의 손실 함수 값들의 합을 지표로 삼는다는 것.

### N개 데이터에 대한 Cross Entropy
<img src="img/deep_learning_images/e_4.3.png" width=320 height=320>

$t_{nk}$는 n번째 데이터의 k번째 값.(정답 레이블)  
$y_{nk}$는 신경망의 출력.  
N으로 나눔으로써 정규화, 평균 손실 함수를 구하는 것이다.

### 미니 배치 학습을 추가.
MNIST의 경우 학습 데이터가 60000개였다. 모든 데이터를 대상으로 손실 함수의 합을 구하려면 시간이 걸린다.  
이 많은 데이터를 대상으로 일일이 손실함수를 계산하는 것은 현실적이지 못하므로, 데이터의 일부를 추려 전체의 근사치로 이용.  
60000장의 훈련 데이터 중 100장을 무작위로 뽑아 이용하는 것이다.

In [1]:
from mnist import load_mnist

x_train, y_train, x_test, y_test = load_mnist(pickle_path="/data/Datasets/MNIST/mnist.pkl")

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 784) (60000, 10)
(10000, 784) (10000, 10)


In [3]:
# 이 중 10장만 무작위로 빼내려면
import numpy as np

batch_size = 10
batch_mask = np.random.choice(x_train.shape[0], batch_size)
batch_mask

array([59584, 43715, 27071, 20377, 19731, 49639, 19382, 14049,  2706,
       25875])

In [7]:
x_batch = x_train[batch_mask]
y_batch = y_train[batch_mask]

print(x_batch.shape, y_batch.shape)
print(y_batch[0])
print(y_batch[1])

(10, 784) (10, 10)
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [5]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size